In [2]:
%pip install pandas==2.0.3
%pip install transformers==4.6.0 
%pip install tensorflow==2.15.0
%pip install tokenizers==0.10.2

Note: you may need to restart the kernel to use updated packages.



     ---------------------------------------- 0.0/70.3 kB ? eta -:--:--
     ---------------------------------- ----- 61.4/70.3 kB 3.2 MB/s eta 0:00:01
     ---------------------------------------- 70.3/70.3 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   - -------------------------------------- 0.1/4.0 MB 5.8 MB/s eta 0:00:01
   - -------------------------------------- 0.2/4.0 MB 2.3 MB/s eta 0:00:02
   ---- ----------------------------------- 0.4/4.0 MB 3.9 MB/s eta 0:00:01
   ------ --------------------------------- 0.6/4.0 MB 3.9 MB/s eta 0:00:01
   --------- ------------------------------ 0.9/4.0 MB 4.6 MB/s eta 0:00:01
   ----------- ---------------------------- 1.1/4.0 MB 4.5 MB/s eta 0:00:01
   ---------------- ----------------------- 1.6/4.0 MB 5.1 MB/s eta 0:00:01
   ------------------ --------------------- 1.9/4.0 MB 5.2 MB/s eta 0:00:01
   -------------------- ------------------- 2.1/4.0 MB 5.1 MB/s eta 0:00:01
   -----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.2.0 requires sentence-transformers<3.0.0,>=2.6.1, which is not installed.
chromadb 0.4.24 requires tokenizers>=0.13.2, but you have tokenizers 0.12.1 which is incompatible.
llama-index-llms-huggingface 0.1.4 requires huggingface-hub<0.21.0,>=0.20.3, but you have huggingface-hub 0.23.0 which is incompatible.
llama-index-llms-huggingface 0.1.4 requires transformers[torch]<5.0.0,>=4.37.0, but you have transformers 4.18.0 which is incompatible.


In [32]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [33]:
import pandas as pd

In [34]:
df = pd.read_csv("./data/SMSSpamCollection",sep="\t",names=["labels","messages"])
df.head(5)

,labels,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [35]:
df.shape

(5572, 2)

In [36]:
# independent features
x= list(df["messages"])

In [37]:
y = list(df["labels"])

In [38]:
y=list(pd.get_dummies(y,drop_first=True)["spam"])

In [39]:
## train test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train,Y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [40]:
from transformers import DistilBertTokenizer, DistilBertModel, AutoTokenizer

# tokenizer = DistilBertTokenizer.from_pretrained('openai-community/gpt2')
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2",pad_token="abc")

In [41]:
train_encodings = tokenizer(X_train,truncation=True,padding=True)
test_encodings = tokenizer(X_test,truncation=True,padding=True)

In [42]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),Y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),Y_test
))

In [43]:
train_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(257,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(257,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.bool, name=None))>

In [44]:
from transformers import TFTrainingArguments
training_args = TFTrainingArguments(
    output_dir='./tmp/results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./tmp/logs',            # directory for storing logs
    logging_steps=10,
)

In [52]:
from transformers import TFDistilBertForSequenceClassification,GPT2ForTokenClassification,GPT2ForSequenceClassification

with training_args.strategy.scope():
    model = GPT2ForSequenceClassification.from_pretrained("openai-community/gpt2")
model

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [53]:
from transformers import TFTrainer, TRainer

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

trainer.train()

u:\Files\LamaIndex\.venv\lib\site-packages\transformers\trainer_tf.py:108: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  warnings.warn(


ValueError: `Checkpoint` was expecting model to be a trackable object (an object derived from `Trackable`), got GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=F

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.predict(test_dataset)

In [ ]:
trainer.predict(test_dataset)[1].shape

In [ ]:
output=trainer.predict(test_dataset)[1]

In [ ]:
from sklearn.metrics import confusion_matrix

cm=confusion_matrix(y_test,output)
cm

In [ ]:
trainer.save_model('senti_model')